# Imports

In [1]:
import datetime
import json
from time import time
import os
from collections import defaultdict
import pickle
from tqdm import tqdm

# from functools import partial
# from itertools import chain, islice
# from multiprocessing import Pool, cpu_count
from pathlib import Path

# from pprint import pprint

import dateutil.parser
import findspark
import lmdb
import numpy as np
import psycopg
import pyspark.sql.functions as sf
from ase.atoms import Atoms
from ase.io.cfg import read_cfg
from dotenv import load_dotenv
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    ArrayType,
    BooleanType,
    DoubleType,
    FloatType,
    IntegerType,
    LongType,
    StringType,
    StructField,
    StructType,
    TimestampType,
)
from colabfit.tools.schema import (
    property_object_schema,
    config_df_schema,
    config_schema,
    property_object_df_schema,
)
from colabfit.tools.configuration import AtomicConfiguration, config_schema
from colabfit.tools.database import DataManager, PGDataLoader
from colabfit.tools.dataset import Dataset, dataset_schema
from colabfit.tools.property import Property, property_object_schema
from colabfit.tools.property_definitions import (
    atomic_forces_pd,
    cauchy_stress_pd,
    potential_energy_pd,
)
from colabfit.tools.schema import configuration_set_schema
import pyarrow as pa

with open("formation_energy.json", "r") as f:
    formation_energy_pd = json.load(f)
findspark.init()
format = "jdbc"
load_dotenv("./.env")

True

# Connect to DB and run loader

In [23]:
JARFILE = os.environ.get("CLASSPATH")
spark = (
    SparkSession.builder.appName("PostgreSQL Connection with PySpark")
    .config("spark.jars", JARFILE)
    .getOrCreate()
)
url = "jdbc:postgresql://localhost:5432/colabfit"
user = os.environ.get("PGS_USER")
password = os.environ.get("PGS_PASS")
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver",
}
loader = PGDataLoader(appname="colabfit", env="./.env")

24/05/21 14:49:53 WARN Utils: Your hostname, arktos resolves to a loopback address: 127.0.1.1; using 172.24.21.25 instead (on interface enp5s0)
24/05/21 14:49:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/05/21 14:49:54 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/21 14:49:55 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


df.rdd  
you can only parallelize one time so don't try to do a dataframe select from an rdd  
updating to sdk 5.1 in a couple weeks  
boto3 and s3 are the amazon file system interactions, mostly for adding metadata TO FILES (not to the database) and interacting with the files as FileExistsError. 
Make sure to spark.stop() at end of  python file.

In [29]:
def update_co_rows_cs_id(self, co_ids: list[str], cs_id: str):
    with psycopg.connect(
        """dbname=colabfit user=%s password=%s host=localhost port=5432"""
        % (
            user,
            password,
        )
    ) as conn:
        # dbname=self.database_name,
        # user=self.properties["user"],
        # password=self.properties["password"],
        # host="localhost",
        # port="5432",
        cur = conn.execute(
            """UPDATE configurations
                SET configuration_set_ids = 
            """
        )
        cur = conn.execute(
            """UPDATE configurations
                SET configuration_set_ids = concat(%s::text, 
                rtrim(ltrim(replace(configuration_set_ids,%s,''), 
                
                '['),']') || ', ', %s::text)
            WHERE id = ANY(%s)""",
            ("[", f"{cs_id}", f"{cs_id}]", co_ids),
            # ("[", f", {cs_id}", f", {cs_id}]"),
        )
        # cur.fetchall()
        conn.commit()

In [112]:
with psycopg.connect(
    dbname="colabfit",
    user=os.environ.get("PGS_USER"),
    password=os.environ.get("PGS_PASS"),
    host="localhost",
) as conn:
    with conn.cursor() as cur:

        # cur.execute(
        #     "UPDATE configurations SET configuration_set_ids = configuration_set_ids || %(cs_id)s WHERE id = ANY(%(co_ids)s)",
        #     {"cs_id": cs["id"], "co_ids": co_ids},
        # )
        # data = cur.fetchall()
        cur.execute(
            "SELECT * FROM public.configurations WHERE id = ANY(%s)",
            [co_ids],
        )
        data2 = cur.fetchall()
    conn.commit()

# In progress

Upsert appears to be this for postgres:
```
update the_table
    set id = id || array[5,6]
where id = 4;
```
* ~~Check for upsert function from pyspark to concatenate lists of relationships instead of primary key id collision~~
* There is no pyspark-upsert function. Will have to manage this possibly through a different sql-based library
* Written: find duplicates, but convert to access database, not download full dataframe
* I see this being used with batches of hashes during upload: something like
    ``` for batch in batches:
            hash_duplicates = find_duplicates(batch, loader/database)
            hash_duplicates.make_change_to_append_dataset-ids
            hash_duplicates.write-to-database
* Where would be the best place to catch duplicates? Keeping in mind that this might be a bulk operation (i.e. on the order of millions, like with ANI1/ANI2x variations)

In [9]:
JARFILE = os.environ.get("CLASSPATH")
spark = (
    SparkSession.builder.appName("PostgreSQL Connection with PySpark")
    .config("spark.jars", JARFILE)
    .getOrCreate()
)
url = "jdbc:postgresql://localhost:5432/colabfit"
user = os.environ.get("PGS_USER")
password = os.environ.get("PGS_PASS")
properties = {
    "user": user,
    "password": password,
    "driver": "org.postgresql.Driver",
}
loader = PGDataLoader(appname="colabfit", env="./.env")

24/05/30 09:52:06 WARN Utils: Your hostname, arktos resolves to a loopback address: 127.0.1.1; using 172.24.21.25 instead (on interface enp5s0)
24/05/30 09:52:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/05/30 09:52:06 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/30 09:52:08 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [ ]:
def _gather_co_po_rows(
    prop_defs: list[dict],
    prop_map: dict,
    dataset_id,
    configs: list[AtomicConfiguration],
):
    """Convert COs and DOs to Spark rows."""
    co_po_rows = []
    for config in configs:
        config.set_dataset_id(dataset_id)
        property = Property.from_definition(
            definitions=prop_defs,
            configuration=config,
            property_map=prop_map,
            standardize_energy=True,
        )
        yield property


from colabfit.tools.property import Property
from colabfit.tools.property_definitions import *


propgen = _gather_co_po_rows(dm.prop_defs, dm.prop_map, dm.dataset_id, dm.configs)
p = next(propgen)

In [ ]:
from importlib import reload

import colabfit.tools.utilities
import colabfit.tools.dataset
import colabfit.tools.database
import colabfit.tools.configuration
import colabfit.tools.configuration_set
import colabfit.tools.property
import colabfit.tools.schema

reload(colabfit.tools.utilities)
reload(colabfit.tools.dataset)
reload(colabfit.tools.configuration)
reload(colabfit.tools.database)
reload(colabfit.tools.property)
reload(colabfit.tools.schema)
AtomicConfiguration = colabfit.tools.configuration.AtomicConfiguration
DataManager = colabfit.tools.database.DataManager
SparkDataLoader = colabfit.tools.database.SparkDataLoader
ConfigurationSet = colabfit.tools.configuration_set.ConfigurationSet
Dataset = colabfit.tools.dataset.Dataset
Property = colabfit.tools.property.Property
dataset_df_schema = colabfit.tools.schema.dataset_df_schema
dataset_schema = colabfit.tools.schema.dataset_schema

#################################################

batches = dm.gather_co_po_in_batches_no_pool()
batch = next(batches)
cos, pos = zip(*batch)
from colabfit.tools.schema import *
import pyspark.sql.functions as sf

podf = spark.createDataFrame(pos, schema=property_object_df_schema).limit(100)
from colabfit.tools.utilities import stringify_df_val

schema = property_object_df_schema
string_cols = [f.name for f in schema if f.dataType.typeName() == "array"]
string_col_udf = sf.udf(stringify_df_val, StringType())

In [ ]:
spark.sql("drop table if exists ndb.colabfit.dev.test_co3")
spark.sql("drop table if exists ndb.colabfit.dev.test_po3")
spark.sql("drop table if exists ndb.colabfit.dev.test_cs3")
spark.sql("drop table if exists ndb.colabfit.dev.test_ds3")

loader.read_table(loader.dataset_table).show(truncate=False)

In [ ]:
spark.sql("show tables in ndb.colabfit.dev").show()
spark.sql("drop table ndb.colabfit.dev.gpw_test_configs")
spark.sql("drop table ndb.colabfit.dev.gpw_test_prop_objects")
spark.sql("drop table ndb.colabfit.dev.gpw_test_config_sets")
spark.sql("drop table ndb.colabfit.dev.gpw_test_datasets")

In [ ]:
from colabfit.tools.utilities import *
from colabfit.tools.database import *
from colabfit.tools.schema import *
from functools import partial

self = loader
batches = dm.gather_co_po_in_batches_no_pool()
batch = next(batches)

co_rows, po_rows = list(zip(*batch))
codf = loader.spark.createDataFrame(co_rows, schema=config_df_schema)
podf = loader.spark.createDataFrame(po_rows, schema=property_object_df_schema)
df = podf
string_cols = [f.name for f in df.schema if f.dataType.typeName() == "array"]
string_col_udf = sf.udf(stringify_df_val, StringType())
for col in string_cols:
    df = df.withColumn(col, string_col_udf(sf.col(col)))

column_name = "atomic_forces_00"


####################################################################################
def get_max_string_length(df, column_name):
    return (
        df.select(column_name)
        .select(sf.length(column_name).alias("string_length"))
        .agg(sf.max("string_length"))
        .collect()[0][0]
    )


def split_long_string_cols(df, column_name: str, max_string_length: int = 60000):
    """
    Splits a long string column into multiple columns based on a maximum string length.
    :param df: Input DataFrame with array cols already stringified
    :param column_name: Name of the column containing the long string
    :param max_string_length: Maximum length for each split string
    :return: DataFrame with the long string split across multiple columns
    """
    if get_max_string_length(df, column_name) <= max_string_length:
        print("no columns truncated")
        return df
    print("columns truncated")
    overflow_columns = [
        f"{'_'.join(column_name.split('_')[:-1])}_{i+1:02}" for i in range(19)
    ]
    if not all([col in df.columns for col in overflow_columns]):
        raise ValueError("Overflow columns not found in target DataFrame schema")
    all_columns = [column_name] + overflow_columns
    tmp_columns = [f"{col_name}_tmp" for col_name in all_columns]
    df = df.withColumn("total_length", sf.length(sf.col(column_name)))
    substring_exprs = [
        sf.when(
            sf.length(sf.col(column_name)) - (i * max_string_length) > 0,
            sf.substring(
                sf.col(column_name), (i * max_string_length + 1), max_string_length
            ),
        )
        .otherwise(sf.lit(None))
        .alias(col_name)
        for i, col_name in enumerate(tmp_columns)
    ]
    df = df.select("*", *substring_exprs)
    for tmp_col, col in zip(tmp_columns, all_columns):
        df = df.drop(col).withColumnRenamed(f"{tmp_col}", col)
    df = df.drop("total_length")
    return df


# Make this to replace the columns, not just add duplicate names
def split_long_string(df, col_name, thresh):
    columns = [sf.col(c) for c in df.columns]
    num_splits = NSITES_COL_SPLITS
    split_exprs = [
        sf.when(
            sf.col(col_name).substr(i * thresh + 1, thresh) != "",
            sf.col(col_name).substr(i * thresh + 1, thresh),
        ).otherwise(sf.lit(""))
        for i in range(num_splits)
    ]
    for i, expr in enumerate(split_exprs):
        columns.append(expr.alias(f"{'_'.join(col_name.split('_')[:-1])}_{i:02}"))
    return df.select(columns)


df1 = split_long_string(podf, "atomic_forces_00", 500 // 19)

In [ ]:
import pyspark.sql.functions as sf

ds = spark.table("ndb.colabfit.ingest.po")
ds = ds.withColumn("energy_conjugate_with_atomic_forces_unit", sf.lit(1))
ds = ds.withColumn("energy_conjugate_with_atomic_forces", sf.lit(1))
for col in [
    "potential_energy_per_atom",
    "free_energy_reference_unit",
    "potential_energy_reference_unit",
    "potential_energy",
    "formation_energy_reference",
    "old_id",
    "potential_energy_reference",
    "potential_energy_unit",
    "atomization_energy_per_atom",
    "atomization_energy_reference",
    "formation_energy_per_atom",
    "electronic_band_gap_property_id",
    "free_energy_unit",
    "free_energy_reference",
    "free_energy",
    "free_energy_per_atom",
    "atomization_energy_reference_unit",
    "adsorption_energy_reference",
    "formation_energy_reference_unit",
    "adsorption_energy_per_atom",
    "adsorption_energy_reference_unit",
]:
    ds = ds.drop(col)
ds.write.mode("append").saveAsTable("ndb.colabfit.dev.pos_from_ingest_large")


loader.config_table = "ndb.colabfit.ingest.co"
co = loader.read_table(loader.config_table)
ds_ids = [
    x["id"]
    for x in spark.read("ndb.colabfit.ingest.ds").limit(10).select("id").collect()
]
id_regexes = "(" + ")|(".join([f".*{ds_id}.*" for ds_id in ds_ids]) + ")"
co1 = co.filter(sf.col("dataset_ids").rlike(id_regexes)).count()

In [ ]:
with loader.session.transaction() as tx:
    table = tx.bucket("colabfit").schema("dev").table("ds_from_ingest")
    schema = pa.schema(
        [
            pa.field("energy_conjugate_with_atomic_forces_mean", pa.float64()),
        ]
    )
    table.add_column(schema)